In [13]:
import numpy as np
import cv2
from skimage.segmentation import slic
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.linear_model import LinearRegression

# Load the pre-trained InceptionV3 model with ImageNet weights
model = InceptionV3(weights='imagenet')

# Function to preprocess image for InceptionV3
def preprocess_image(image):
    image = cv2.resize(image, (299, 299))
    image = image.astype('float32')
    image = preprocess_input(image)
    return image

# Function to generate superpixels
def generate_superpixels(image, n_segments=50):
    segments = slic(image, n_segments=n_segments, compactness=10)
    return segments

# Function to create perturbed images
def create_perturbed_images(image, segments, num_perturbations=100):
    perturbed_images = []
    for _ in range(num_perturbations):
        perturbed_image = image.copy()
        num_superpixels_to_black = np.random.randint(1, np.max(segments))
        superpixels_to_black = np.random.choice(np.unique(segments), num_superpixels_to_black, replace=False)
        for superpixel in superpixels_to_black:
            perturbed_image[segments == superpixel] = 0
        perturbed_images.append(perturbed_image)
    return perturbed_images

# Function to make predictions
def make_predictions(images):
    predictions = []
    for img in images:
        preprocessed_img = preprocess_image(img)
        pred = model.predict(np.expand_dims(preprocessed_img, axis=0))
        predictions.append(pred)
    return np.array(predictions)

# Function to fit a simple interpretable model
def fit_interpretable_model(X, y):
    model = LinearRegression()
    model.fit(X, y)
    return model

# Load and preprocess the input image
image_path = 'Image/cat.jpg'  # Replace with your image path
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"Image not found at the path: {image_path}")

# Generate superpixels
segments = generate_superpixels(image)

# Create perturbed images
perturbed_images = create_perturbed_images(image, segments, num_perturbations=100)

# Make predictions for perturbed images
perturbed_predictions = make_predictions(perturbed_images)

# Flatten the predictions for interpretable model
perturbed_predictions_flat = perturbed_predictions.reshape(len(perturbed_predictions), -1)

# Prepare X and y for interpretable model
X = perturbed_predictions_flat
y = np.ones(len(perturbed_predictions))  # Assuming all perturbed images are of the same class

# Fit the interpretable model
interpretable_model = fit_interpretable_model(X, y)

# Get the coefficients (importance) of each feature (superpixel)
importances = interpretable_model.coef_

# Sort superpixels by importance
sorted_superpixels = np.argsort(np.abs(importances))

# Display top 5 important superpixels
num_superpixels_to_plot = 5
top_superpixels = sorted_superpixels[-num_superpixels_to_plot:]

# Print the top 5 superpixels and their importance scores
print("Top 5 important superpixels:")
for i, superpixel_index in enumerate(top_superpixels):
    importance_score = importances[superpixel_index]
    print(f"Superpixel {superpixel_index}: Importance Score = {importance_score:.4f}")


1/1 [==============================] - 0s 116ms/step
Top 5 important superpixels:
Superpixel 339: Importance Score = 0.0000
Superpixel 340: Importance Score = 0.0000
Superpixel 341: Importance Score = 0.0000
Superpixel 328: Importance Score = 0.0000
Superpixel 999: Importance Score = 0.0000
